### Load Data

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('/gdrive/My Drive/AI-ML/data/International-airline-passengers.csv', index_col='Month')
df.sort_index(inplace=True)
df.head()

In [0]:
df.shape

In [0]:
df.plot()

### Data Preprocessing

In [0]:
#Check for null values
df.isnull().sum()

In [0]:
#Drop NA
df.dropna(inplace=True)

In [0]:
#Check Data Range
print('Min', np.min(df))
print('Max', np.max(df))

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
#Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(df)

### Split Data

In [0]:
#70% examples will used for training (in the begining)
train_size = int(len(scaled) * 0.70)

#30% will be used for Test
test_size = len(scaled - train_size)

#Split the data
train, test = scaled[0:train_size, :], scaled[train_size: len(scaled), :]
print('train: {}\ntest: {}'.format(len(train), len(test)))

In [0]:
train.shape

### Build Input and Output data
for both Training and Test

In [0]:
#window - how long the sequence will be
def create_dataset(dataset, window=1):
    
    dataX, dataY = [], []
    
    for i in range(len(dataset)-window):
        
        a = dataset[i:(i+window), 0]        
        dataX.append(a)
        dataY.append(dataset[i + window, 0])
    
    return np.array(dataX), np.array(dataY)

In [0]:
#Create Input and Output
window_size = 2
X_train, y_train = create_dataset(train, window_size)
X_test, y_test = create_dataset(test, window_size)

In [0]:
#Make it 3 Dimensional Data - needed for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
print(X_train.shape)
print(X_test.shape)

### Build the Model

In [0]:
import tensorflow as tf

In [0]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(32, input_shape=(window_size, 1)))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

In [0]:
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), batch_size=32)

### Visualize Prediction
For both training and Test Data

In [0]:
#Get prediction for both Training and Test Data
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)

In [0]:
#Un-normalize the predited data
trainPredict = scaler.inverse_transform(trainPredict)
testPredict = scaler.inverse_transform(testPredict)

In [0]:
import matplotlib.pyplot as plt

In [0]:
trainPredictPlot = np.empty_like(scaled)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[window_size:len(trainPredict)+window_size, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(scaled)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(window_size*2):len(scaled), :] = testPredict
# plot baseline and predictions
plt.figure(figsize=(20,10))
plt.plot(scaler.inverse_transform(scaled))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()